In [15]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torchvision
import math
from sklearn.preprocessing import StandardScaler
from thermography_dataset_one_layer import ThermDataset
from torch.utils.data import DataLoader
import seaborn as sns

In [16]:
dataset = ThermDataset(fp='combined_data.xlsx', noise_scale=0, layer=False)

In [17]:
dataloader = DataLoader(dataset, shuffle=True, batch_size=32)

In [18]:
for xbatch, ybatch in dataloader:
    print(xbatch)

tensor([[8.8277e-13],
        [5.0948e-12],
        [3.5592e-12],
        [9.6392e-13],
        [4.1340e-12],
        [2.8909e-12],
        [4.2744e-12],
        [3.3936e-12],
        [3.9870e-12],
        [3.3090e-12],
        [1.4003e-12],
        [3.5521e-12],
        [1.0491e-12],
        [1.0753e-12],
        [2.5875e-12],
        [1.2177e-12],
        [3.1458e-12],
        [2.6345e-12],
        [1.2452e-12],
        [1.9175e-12],
        [1.3351e-12],
        [1.1600e-12],
        [3.6951e-12],
        [9.7862e-13],
        [3.1016e-12],
        [2.0825e-12],
        [2.8266e-12],
        [2.7346e-12],
        [2.1844e-12],
        [1.9643e-12],
        [3.5913e-12],
        [1.1301e-12]])
tensor([[2.5088e-12],
        [1.8868e-12],
        [1.9406e-12],
        [1.5839e-12],
        [3.8343e-12],
        [3.9502e-12],
        [2.3088e-12],
        [2.8567e-12],
        [3.2422e-12],
        [4.8574e-12],
        [3.3858e-12],
        [4.0313e-12],
        [3.1720e-12],
        [

In [19]:
dataset = ThermDataset('combined_data.xlsx', noise_scale=0)

TypeError: ThermDataset.__init__() missing 1 required positional argument: 'layer'

In [ ]:
dataset.x

tensor([306.8098, 301.7977, 327.9540,  ..., 362.1555, 350.2216, 344.9117])

In [ ]:
for num in range(1,12):
    print(num)

1
2
3
4
5
6
7
8
9
10
11
